# MET's Institute of Engineering
# Department of Computer Engineering
# Academic Year 2021-22

## A MINI PROJECT REPORT ON ‘Student Performance in Exam’

Submitted by
> Mr. Shivam More (Roll No: 20)<br>
> Mr. Tejas Thorat (Roll No: 44)<br>
> Mr. Nikhil Jadhav (Roll No: 60)

Under the guidance of
> Prof. Vishal Patil

For the Subject
> Laboratory Practice II (410247) -
> Data Mining and Warehousing (410244 (D))

Submitted in partial fulfilment of the requirements for the award of the 
>Degree of Bachelor in Computer Engineering

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.svm import SVC


In [8]:
df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [9]:
df.shape

(45000, 16)

In [10]:
test_df.shape

(13107, 15)

In [11]:
dt = df
test_dt = test_df

In [12]:
df['Elo'] = df['Elo'].fillna(df['Elo'].mode()[0])
df['opp_Elo'] = df['opp_Elo'].fillna(df['opp_Elo'].mode()[0])
df['win_equivalent'] = df['win_equivalent'].fillna(df['win_equivalent'].mode()[0])

test_df['Elo'] = test_df['Elo'].fillna(test_df['Elo'].mode()[0])
test_df['opp_Elo'] = test_df['opp_Elo'].fillna(test_df['opp_Elo'].mode()[0])
test_df['win_equivalent'] = test_df['win_equivalent'].fillna(test_df['win_equivalent'].mode()[0])

# df=df.fillna(df.mean())

In [13]:
lb = LabelEncoder()
df['team_id'] = lb.fit_transform(df['team_id'])
df['opp_team_id'] = lb.fit_transform(df['opp_team_id'])

test_df['team_id'] = lb.fit_transform(test_df['team_id'])
test_df['opp_team_id'] = lb.fit_transform(test_df['opp_team_id'])


In [14]:
new = df["date"].str.split("/", n = 2, expand = True) 
  
df["date1"]= new[0] 
df["month"]= new[1] 
df["year"]= new[2] 

# Dropping old Name columns 
df = df.drop(['date'], axis=1)




new1 = test_df["date"].str.split("/", n = 2, expand = True) 
  
test_df["date1"]= new1[0] 
test_df["month"]= new1[1] 
test_df["year"]= new1[2] 

# Dropping old Name columns 
test_df = test_df.drop(['date'], axis=1)


In [15]:
df['playoff'] = lb.fit_transform(df['playoff'])
test_df['playoff'] = lb.fit_transform(test_df['playoff'])

In [16]:
df['date1'] = df['date1'].astype(int)
df['month'] = df['month'].astype(int)
df['year'] = df['year'].astype(int)
test_df['date1'] = test_df['date1'].astype(int)
test_df['month'] = test_df['month'].astype(int)
test_df['year'] = test_df['year'].astype(int)

In [17]:
df.head()

,Id,game_seq,season_end,season_game_seq,playoff,team_id,Elo,opp_team_id,opp_Elo,win_equivalent,bet_ratio,home_crowd,opp_crowd,total_crowd,game_result,date1,month,year
0,1,39323,1997,2,0,18,1511.6364,2,1300.0000,18.998001,0.616437,6600,4800,11400,1,11,2,1996
1,2,59211,2012,87,1,15,1690.4612,37,1732.2164,59.721931,0.583044,5600,4300,9900,0,6,17,2012
2,3,60691,2014,12,0,10,1550.2258,3,1300.0000,18.998001,0.606468,8600,3800,12400,0,11,20,2013
3,4,31900,1990,69,0,4,1300.0000,9,1629.6985,39.305004,0.390694,7200,4000,11200,0,3,29,1990
4,5,48134,2004,30,0,20,1300.0000,4,1300.0000,28.311192,0.492179,8600,4900,13500,0,1,2,2004


In [18]:
test_df.head()

,Id,game_seq,season_end,season_game_seq,playoff,team_id,Elo,opp_team_id,opp_Elo,win_equivalent,bet_ratio,home_crowd,opp_crowd,total_crowd,date1,month,year
0,45001,50929,2006,45,0,6,1452.0272,4,1467.3956,36.577610,0.619442,8700,3300,12000,2,1,2006
1,45002,62745,2015,60,0,7,1498.2162,1,1283.8668,40.762897,0.859306,7000,2600,9600,3,4,2015
2,45003,53291,2008,28,0,59,1495.3258,6,1392.2981,40.791542,0.762914,3800,1500,5300,12,25,2007
3,45004,24991,1984,18,0,58,1602.1180,11,1528.8450,52.006989,0.730555,7000,300,7300,12,2,1983
4,45005,18807,1977,73,0,23,1481.2228,4,1571.5261,29.636082,0.513951,4900,1000,5900,3,25,1977


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               45000 non-null  int64  
 1   game_seq         45000 non-null  int64  
 2   season_end       45000 non-null  int64  
 3   season_game_seq  45000 non-null  int64  
 4   playoff          45000 non-null  int64  
 5   team_id          45000 non-null  int32  
 6   Elo              45000 non-null  float64
 7   opp_team_id      45000 non-null  int32  
 8   opp_Elo          45000 non-null  float64
 9   win_equivalent   45000 non-null  float64
 10  bet_ratio        45000 non-null  float64
 11  home_crowd       45000 non-null  int64  
 12  opp_crowd        45000 non-null  int64  
 13  total_crowd      45000 non-null  int64  
 14  game_result      45000 non-null  int64  
 15  date1            45000 non-null  int32  
 16  month            45000 non-null  int32  
 17  year        

In [20]:
test_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13107 entries, 0 to 13106
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               13107 non-null  int64  
 1   game_seq         13107 non-null  int64  
 2   season_end       13107 non-null  int64  
 3   season_game_seq  13107 non-null  int64  
 4   playoff          13107 non-null  int64  
 5   team_id          13107 non-null  int32  
 6   Elo              13107 non-null  float64
 7   opp_team_id      13107 non-null  int32  
 8   opp_Elo          13107 non-null  float64
 9   win_equivalent   13107 non-null  float64
 10  bet_ratio        13107 non-null  float64
 11  home_crowd       13107 non-null  int64  
 12  opp_crowd        13107 non-null  int64  
 13  total_crowd      13107 non-null  int64  
 14  date1            13107 non-null  int32  
 15  month            13107 non-null  int32  
 16  year             13107 non-null  int32  
dtypes: float64(4

In [21]:
df.shape

(45000, 18)

In [22]:
test_df.shape


(13107, 17)

In [23]:
df.drop('Id', axis=1, inplace=True)
df.drop('playoff', axis=1, inplace=True)
# df.drop('season_end', axis=1, inplace=True)
df.drop('date1', axis=1, inplace=True)
# df.drop('year', axis=1, inplace=True)
df.drop('team_id', axis=1, inplace=True)
df.drop('opp_team_id', axis=1, inplace=True)
df.drop('season_end', axis=1, inplace=True)
df.drop('year', axis=1, inplace=True)


test_df.drop('Id', axis=1, inplace=True)
test_df.drop('playoff', axis=1, inplace=True)
# test_df.drop('season_end', axis=1, inplace=True)
test_df.drop('date1', axis=1, inplace=True)
# test_df.drop('year', axis=1, inplace=True)
test_df.drop('team_id', axis=1, inplace=True)
test_df.drop('opp_team_id', axis=1, inplace=True)
test_df.drop('season_end', axis=1, inplace=True)
test_df.drop('year', axis=1, inplace=True)

In [24]:
df.shape

(45000, 11)

In [25]:
test_df.shape

(13107, 10)

In [26]:
y = df['game_result']
df.drop('game_result', axis=1, inplace=True)
x = df

In [27]:
x.shape

(45000, 10)

In [28]:
y.shape

(45000,)

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size = 0.1, random_state = 0)

In [30]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier(n_estimators=600)
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
acc_randomforest = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_randomforest)

77.4


In [31]:
from sklearn import metrics

print(metrics.confusion_matrix(y_val, y_pred))

[[2444  331]
 [ 686 1039]]


In [32]:
# importances = randomforest.feature_importances_
print(randomforest.feature_importances_)

[0.09884209 0.07507408 0.13240455 0.09963422 0.15160131 0.17627526
 0.06697976 0.06675681 0.07013566 0.06229626]


In [33]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
acc_gaussian = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_gaussian)


66.56


In [34]:
print(metrics.confusion_matrix(y_val, y_pred))

[[2258  517]
 [ 988  737]]


In [35]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
acc_logreg = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_logreg)

65.47


In [36]:
print(metrics.confusion_matrix(y_val, y_pred))

[[2358  417]
 [1137  588]]


In [37]:
# Support Vector Machines
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_val)
acc_svc = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_svc)

61.67


In [38]:
print(metrics.confusion_matrix(y_val, y_pred))

[[2775    0]
 [1725    0]]


In [39]:
# Linear SVC
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_val)
acc_linear_svc = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_linear_svc)

61.58


C:\Users\jadha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [40]:
print(metrics.confusion_matrix(y_val, y_pred))

[[2672  103]
 [1626   99]]


In [41]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier(class_weight=None, max_depth=32,
 max_features=None, max_leaf_nodes=None,
 min_impurity_decrease=1e-07, min_samples_leaf=5,
 min_samples_split=2, min_weight_fraction_leaf=0.0,
 random_state=None)
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
acc_decisiontree = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_decisiontree)

73.11


In [42]:
print(metrics.confusion_matrix(y_val, y_pred))

[[2220  555]
 [ 655 1070]]


In [43]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier(n_estimators=2000)
gbk.fit(x_train, y_train)
y_pred = gbk.predict(x_val)
acc_gbk = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_gbk)

83.62


In [44]:
print(metrics.confusion_matrix(y_val, y_pred))

[[2520  255]
 [ 482 1243]]


In [45]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines',  
              'Random Forest', 'Naive Bayes', 'Linear SVC',
              'Decision Tree', 'Gradient Boosting Classifier'],
    'Score': [acc_svc,  
              acc_randomforest, acc_gaussian, acc_linear_svc, acc_decisiontree,
               acc_gbk]})
models.sort_values(by='Score', ascending=False)

,Model,Score
5,Gradient Boosting Classifier,83.62
1,Random Forest,77.40
4,Decision Tree,73.11
2,Naive Bayes,66.56
0,Support Vector Machines,61.67
3,Linear SVC,61.58
